<a href="https://colab.research.google.com/github/xbeafraid1/ILTAL/blob/main/Python_GBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#지금 만들어야 하는 모델 : 
CatBoost, AdaBoost, XGBoost, LGBM 중 어느 것을 쓸 지 아직 정해지지 않았지만 공통적으로 적용될 수 있는 모델을 만들어보려고 함.



### 5일, 4일, 3일, 2일, 1일 중 어느 것으로 앞의 것들을 예측해볼 지는 아직 정해지지 않음.


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import time
import torch
import statsmodels.api as sm
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.regression.quantile_regression import QuantReg
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from pandas import DataFrame, concat
import torch
import os

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv') 

In [ ]:
# !cp를 이용하여 기존에 있었던 함수를 불러온다.
# GB_functions.ipynb에 전처리 함수를 저장해 놓았으므로 얘를 JY로 불러온다.
!cp /content/drive/MyDrive/Jupyter/unlimited_power/working__/GB_functions.ipynb .
import GB_functions as JY

# 불러온 데이터를 GB_functions에 미리 정의해 둔 data_loadder 함수에 때려 넣으면
# 자동으로 train_X, train_y, test_X, test_y 4가지가 선언된다.
train_set, test_set = MJ.preprocessing(raw_data)

In [ ]:
# Hour와 Minute 변수를 활용하여 Time 변수 추가함.

def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset = dataset[what_to_use]
  return dataset

In [ ]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 함수 (series 함수를 supervised 형태로 바꾸어주는 함수) - Multivariable을 이용한 Gradient Boost 모델에서도 필요함.
def series_to_supervised(data, n_in=1, n_out=1, target = 'TARGET', dropnan=True):
    df = DataFrame(data)
    df.drop(target, axis = 1, inplace=True)
    df2 = DataFrame(data[target])
    cols, names = list(), list()
    n_vars = 1 if type(df) is list else df.shape[1]
    n_vars2 = 1 if type(df2) is list else df2.shape[1]
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df2.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
def data_maker(train_X, train_y, test_X, test_y):
  train_X = torch.tensor(np.array(train_X), dtype=torch.float)
  train_X = train_X.transpose(1,0)
  train_y = torch.tensor(np.array(train_y), dtype=torch.float)

  test_X = torch.tensor(np.array(test_X), dtype=torch.float)
  test_X = test_X.transpose(1,0)
  test_y = torch.tensor(np.array(test_y), dtype=torch.float)
  return train_X, train_y, test_X, test_y

In [ ]:
def data_loader(dataset):

  ############ 파라미터 #####################
  # 하루의 틱
  ticks = 48
  # 예측에 사용할 일수
  days = 7
  n_days = ticks*days

  # 미래 예측할 일수
  future_days = 2
  future_window = ticks * future_days

  ### 모든변수
  # ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']
  # 사용할 변수
  what_to_use = ['Time', 'DHI','DNI','RH','T','TARGET']

  n_features = len(what_to_use)
  n_obs = n_days * n_features


  dataset = drop_clms(dataset)
  dataset = series_to_supervised(dataset, n_days, future_window, target='TARGET')
  train_X, train_y, test_X, test_y = seperator(dataset)
  train_X, train_y, test_X, test_y = data_maker(train_X, train_y, test_X, test_y)
  return train_X, train_y, test_X, test_y

In [ ]:
# 시계열 분석의 백테스팅 - walk-forward validation for univariate data
# Cut Point를 정해서 그 전은 Train / 그 이후는 Test Set으로 정함.
# 하나의 Step을 정한 다음에, 예를 들어서 1 week forecast라고 하면 test의 데이터로 1 step 예측 후, 1 step 간 부분의 real data 까지 포함해서 그 다음 부분을 예측하는 방식.
def walk_forward_validation(data, n_test):
	predictions = list()
	# 데이터셋 분리
	train, test = train_test_split(data, n_test)
	# train 데이터셋의 seed 히스토리
	history = [x for x in train]
	# test set을 부분부분 train set에 포함시킴.
	for i in range(len(test)):
		# split test row into input and output columns
		testX, testy = test[i, :-1], test[i, -1]
		# fit model on history and make a prediction - One Step Forecast 수행함.
		yhat = xgboost_forecast(history, testX)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop - 여기서 actual observation을 history에 추가시킴.
		history.append(test[i])
		# summarize progress
		print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
	# estimate prediction error
	error = mean_absolute_error(test[:, -1], predictions)
	return error, test[:, 1], predictions

In [ ]:
# train과 test set으로 분리하는 train_test_split
def train_test_split(data, n_test):
	return data[:-n_test, :], data[-n_test:, :]

In [ ]:
#  XGBRegressor 사용하여 One-step 예측하는 함수를 구축함.

def xgboost_forecast(train, testX):
	# transform list into array
	train = asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1], train[:, -1]
	# fit model
	model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict([testX])
	return yhat[0]


  ## 여기서 더 못나가겠다 무슨 말하는지 모르겠다 

In [ ]:
############ 파라미터 #####################
# 하루의 틱
ticks = 48
# 예측에 사용할 일수
days = 7
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']
# 사용할 변수
what_to_use = ['Time', 'DHI','DNI','RH','T','TARGET']

n_features = len(what_to_use) - 1
n_obs = n_days * n_features

# 한 번에 뭉테기로 투입할 자료의 양
batches = 100
# 몇 번이나 반복하여 학습할 것인다.
epoch = 200

In [ ]:
train_X, train_y, test_X, test_y = data_loader(raw_data)

In [ ]:
type(train_X)

torch.Tensor